# Bebording
Set aan diverse analyses die fouten in het NDW bordenbestand opsporen.

In [1]:
import psycopg2
try:
    postgres_pw = TokenLibrary.getSecret("redacted", "redacted", "redacted")
    conn = psycopg2.connect(dbname="rdt_dev",user="redacted",host="redacted",password=postgres_pw)
    conn.autocommit = True
    cur=conn.cursor()
    print("Database connected")
except:
    print("I am unable to connect to the database")

In [2]:
# subselection_string="('Noord-Holland','Flevoland')"
subselection_string="('Noord-Holland','Flevoland','Utrecht')"

## Tekst-waarde
De kolom 'black-code' uit het NDW bordenbestand dient bij borden van RVV-code C17-C21 een numerieke waarde te bevatten. Deze waarde geeft een hoogte, lengte, breedte of gewicht aan. Deze analyse detecteerd niet-numerieke waarden.

In [3]:
sql='''drop table if exists int_bordenanalyse.tekstwaarde_foutief;
CREATE TABLE int_bordenanalyse.tekstwaarde_foutief AS (
    SELECT      id, ndw_id, rvv_code, black_code
    FROM        stg_ndw_borden.ndw_bebording_raw_catc
    WHERE       rvv_code IN ('C17','C18','C19','C20','C21') 
    AND         (black_code !~ '^[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?$' --regexp voor numerieke waarden
                OR
                black_code IS NULL)
    AND         black_code is distinct from 'ZE'
    ORDER BY    rvv_code, black_code
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'tekstwaarde_foutief'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

## Wegvak koppeling
De kolom 'location.road.wvk_id' bevat geen bruikbare waarde. Op basis van nabijheid wordt een voorstel gedaan, waar mogelijk


In [4]:
# old
sql='''drop table if exists int_bordenanalyse.ndw_wvk_foutief;
CREATE TABLE int_bordenanalyse.ndw_wvk_foutief AS (
SELECT  id as bord_id, ndw_id, geom as bord_geom, rvv_modelnummer,
        CASE WHEN dist<10 THEN wvk_id ELSE NULL END as voorgestelde_wvk_id,
        CASE WHEN dist<10 THEN roadgeom ELSE NULL END as wvk_geom,
        CASE WHEN dist<10 THEN dist ELSE NULL END as wvk_dist
FROM (
    SELECT  *
            ,row_number() OVER (PARTITION BY sub.id ORDER BY dist ASC) AS rn
    FROM    (
        SELECT      b.id, b.ndw_id, b."location.road.name", b."location.county.name", b."location.road.wvk_id",b.geom
                    ,w.wvk_id
                    ,b.rvv_code as rvv_modelnummer
                    ,wd.geom as roadgeom
                    ,b.geom <-> ST_transform(wd.geom,28992) as dist
        FROM        stg_ndw_borden.ndw_bebording_raw_geom_subselection b
        LEFT JOIN   stg_ndw_wegen.nwb_raw_subselection w
        ON          b."location.road.name"=w.stt_naam
        LEFT JOIN   prd_ndw_wegen.nwb_wegvakken_directed wd
        ON          w.wvk_id=wd.id
        WHERE       b."location.road.wvk_id" IS NULL
        ORDER BY    b."location.road.name"
                    , dist
    ) sub
) sub2
WHERE       rn=1
ORDER BY    id,wvk_dist
);'''
# cur.execute(sql)

In [5]:
sql='''drop table if exists int_bordenanalyse.ndw_wvk_foutief;
CREATE TABLE int_bordenanalyse.ndw_wvk_foutief AS (
    WITH bv3 AS (
            SELECT  bord_id, ndw_id, rvv_modelnummer, gemeentenaam, straatnaam, geom
                    ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
                    ,nwb_wegvak_id_volgens_ndw, nwb_wegvak_id_volgens_ndw_directed, _wegvak_id_volgens_script
            FROM    prd_ndw_borden.ndw_bebording_verrijkt_3
            WHERE   rvv_modelnummer IN ('C1','C07A','C07zb','C6','C7','C8','C9','C10','C11','C12','C17','C18','C19','C20','C21','C22','C23')
        )
        ,borden_wvk_dir_null AS (
            SELECT  *
            FROM    bv3
            WHERE   nwb_wegvak_id_volgens_ndw_directed IS NULL
        )
        -- Wel NDW-wegvak, niet te koppelen op richting
        ,borden_windrichting AS (
            SELECT  *
            FROM    borden_wvk_dir_null
            WHERE   nwb_wegvak_id_volgens_ndw IS NOT NULL
        )
        -- Geen NDW-wegvak
        ,borden_wvk_null AS (
            SELECT  *
            FROM    borden_wvk_dir_null
            WHERE   nwb_wegvak_id_volgens_ndw IS NULL
        )
        -- Geen NDW-wegvak, wel via script
        ,borden_script AS (
            SELECT  *
            FROM    borden_wvk_null
            WHERE   _wegvak_id_volgens_script IS NOT NULL
        )
        -- Geen NDW-wegvak, geen voorstel via script
        ,borden_script_null AS (
            SELECT  *
            FROM    borden_wvk_null
            WHERE   _wegvak_id_volgens_script IS NULL
        )
        -- Wel NDW-wegvak, mismatch met script => niet rapporteren maar wel detecteren
        ,borden_script_mismatch AS (
            SELECT  *
            FROM    bv3
            WHERE   nwb_wegvak_id_volgens_ndw IS NOT NULL
            AND     _wegvak_id_volgens_script IS NOT NULL
            AND     nwb_wegvak_id_volgens_ndw::INT != _wegvak_id_volgens_script
        )
        -- Samen is dit de volledige set aan borden die in borden_wvk_dir_null voorkomt
        ,borden_union AS (
            SELECT  *
                    ,'Onbekende_rijrichting' as code
                    ,'Bord niet te koppelen aan rijrichting' as beschrijving
                    ,nwb_wegvak_id_volgens_ndw::INT as voorgestelde_wvk_id
            FROM    borden_windrichting
            UNION
            SELECT  *
                    ,'Onbekende_NWB_ID' as code
                    ,'Geen NWB wegvak ID beschikbaar' as beschrijving
                    ,abs(_wegvak_id_volgens_script::INT) as voorgestelde_wvk_id
            FROM    borden_script
            UNION
            SELECT  *
                    ,'Onbekende_NWB_ID' as code
                    ,'Geen NWB wegvak ID beschikbaar' as beschrijving
                    ,NULL::INT as voorgestelde_wvk_id
            FROM    borden_script_null
        )
    SELECT      bu.*
                ,wd.geom as wvk_geom
    FROM        borden_union as bu
    LEFT JOIN   prd_ndw_wegen.nwb_wegvakken_directed as wd
    ON          bu.voorgestelde_wvk_id::INT = wd.id
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'ndw_wvk_foutief'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

# Helperfunctie voor bereikbaarheidscheck
Checkt of de bereikbaarheidscheck al is gerunt voor een specifieke voertuigsamenstelling. 

Zo ja: 
- rapporteert het resultaat, 

zo nee: 
- runt de check, 
- slaat het resultaat op, en 
- rapporteert het resultaat

In [6]:
wellconnectednode=254441013

In [7]:
sql=f"""
DROP TABLE IF EXISTS int_bordenanalyse.bereikbaarheidsqueries;
CREATE TABLE IF NOT EXISTS int_bordenanalyse.bereikbaarheidsqueries (
    v_type VARCHAR,
    trailer BOOLEAN,
    v_length FLOAT,
    v_width FLOAT,
    v_height FLOAT,
    v_axleload FLOAT,
    v_weight FLOAT,
    road_id int,
    geom GEOMETRY
);
CREATE TABLE IF NOT EXISTS int_bordenanalyse.bereikbaarheidsqueries_default AS (
    SELECT  *
    FROM    prd_conflicten.ST_Bereikbaarheid('''auto''',FALSE,1,1,1,1,1,{wellconnectednode},0)
)""";
cur.execute(sql)

In [8]:
# This function produces identical output to prd_conflicten.ST_Bereikbaarheid(), but re-uses outputs to prevent unnecessary resource use
sql=f'''CREATE OR REPLACE FUNCTION ST_Bereikbaarheidscheck (
    var_startpoint INT, -- DEFAULT 254441013
    var_v_type VARCHAR,
    var_trailer BOOLEAN,
    var_v_length FLOAT,
    var_v_width FLOAT,
    var_v_height FLOAT,
    var_v_axleload FLOAT,
    var_v_weight FLOAT
)
RETURNS TABLE(road_id int,geom GEOMETRY) AS $$
    DECLARE
        test_if_empty INTEGER;
    BEGIN 
        select  q1.road_id
        from    int_bordenanalyse.bereikbaarheidsqueries q1
        into    test_if_empty
        where   v_type=var_v_type AND trailer = var_trailer AND v_length = var_v_length AND v_width = var_v_width AND v_height = var_v_height AND v_axleload = var_v_axleload AND v_weight = var_v_weight;
        
        if not found then
            -- If not present in the table, add to the table
            INSERT INTO int_bordenanalyse.bereikbaarheidsqueries(v_type,trailer,v_length,v_width,v_height,v_axleload,v_weight,road_id,geom)
            SELECT  var_v_type as v_type
                    ,var_trailer as trailer
                    ,var_v_length as v_length
                    ,var_v_width as v_width
                    ,var_v_height as v_height
                    ,var_v_axleload as v_axleload
                    ,var_v_weight as v_weight
                    ,b.id as road_id,b.geom
            FROM    prd_conflicten.ST_Bereikbaarheid(var_v_type,var_trailer,var_v_length,var_v_width,var_v_height,var_v_axleload,var_v_weight,var_startpoint,0) b
            -- prevent table bloat
            WHERE   b.id NOT IN (
                SELECT  id
                FROM    int_bordenanalyse.bereikbaarheidsqueries_default
            );

            -- Ensure there is always at least 1 entry added to the table
            INSERT INTO int_bordenanalyse.bereikbaarheidsqueries(v_type,trailer,v_length,v_width,v_height,v_axleload,v_weight,road_id,geom)
            VALUES  (var_v_type,var_trailer,var_v_length,var_v_width,var_v_height,var_v_axleload,var_v_weight,NULL,NULL);
        end if;
    
        RETURN QUERY
            select  q2.road_id, q2.geom
            from    int_bordenanalyse.bereikbaarheidsqueries q2
            where   v_type=var_v_type AND trailer = var_trailer AND v_length = var_v_length AND v_width = var_v_width AND v_height = var_v_height AND v_axleload = var_v_axleload AND v_weight = var_v_weight;
    END;
$$ LANGUAGE plpgsql;'''
cur.execute(sql)

# Matrixian
Op Matrixian, een NDW logistiek platform, staan gebieden ingetekend waar grootte-/gewichtsbeperkingen zouden moeten gelden. Deze beperkingen zouden overeen moeten komen met de beperkingsborden (categorie C: https://www.verkeersbordenoverzicht.nl/#C).
De checks hieronder berekenen per Matrixian gebied of de daadwerkelijke bereikbaarheid (op basis van verkeersborden) overeenkomt met de gewenste restrictie(s).

Onderstaande scripts gaan er van uit dat "RDT_Bordenanalyse_MatrixianApiCall" gerund heeft/is

In [9]:
# only select entries with start&end datetimes
# recalculate (groups of) entries into their respective "verkeersborden" types (see: https://www.verkeersbordenoverzicht.nl/#C)
sql='''DROP TABLE IF EXISTS int_bordenanalyse.matrixian_vehiclerestrictions_constraints_verkeersborden;
CREATE TABLE int_bordenanalyse.matrixian_vehiclerestrictions_constraints_verkeersborden AS (
    SELECT  "items.id" as id
            ,min(CASE WHEN type='sizeConstraint' AND "sizeConstraint"='vehicleLength' THEN "minimum.value"/100 ELSE 0 END) as C17
            ,min(CASE WHEN type='sizeConstraint' AND "sizeConstraint"='vehicleWidth' THEN "minimum.value"/100 ELSE 0 END) as C18
            ,min(CASE WHEN type='sizeConstraint' AND "sizeConstraint"='vehicleHeight' THEN "minimum.value"/100 ELSE 0 END) as C19
            ,min(CASE WHEN type='weightConstraint' THEN "maximum.value"/1000 ELSE 0 END) as C21
    FROM    stg_bordenanalyse.matrixian_vehiclerestrictions_constraints
    WHERE   "items.id" NOT IN (
                SELECT  "items.id" as id
                FROM    stg_bordenanalyse.matrixian_vehiclerestrictions_constraints
                WHERE   (type='startDateTimeConstraint' AND "startDateTime"::timestamp>(NOW()+'1 year'::interval))
                OR      (type='endDateTimeConstraint' AND "startDateTime"::timestamp<NOW())
            )
    AND NOT (type='weightConstraint' AND type='sizeConstraint' AND type IS NULL)
    AND     (type='weightConstraint' OR type='sizeConstraint')
    GROUP BY id
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'matrixian_vehiclerestrictions_constraints_verkeersborden'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'data cleaning'::VARCHAR);'''
cur.execute(sql_meta)

In [10]:
# Combine matrixian data with all restrictions and the geom in a single row
sql='''DROP TABLE IF EXISTS int_bordenanalyse.matrixian_vehiclerestrictions_verkeersborden_geom;
CREATE TABLE int_bordenanalyse.matrixian_vehiclerestrictions_verkeersborden_geom AS (
    SELECT      mv.* 
                ,CASE WHEN mvcv.C17 IS NULL THEN 0 ELSE mvcv.C17 END as C17
                ,CASE WHEN mvcv.C18 IS NULL THEN 0 ELSE mvcv.C18 END as C18
                ,CASE WHEN mvcv.C19 IS NULL THEN 0 ELSE mvcv.C19 END as C19
                ,CASE 
                    WHEN "administrativeReference.maximumSingleAxleWeight" IS NULL THEN 0 
                    ELSE "administrativeReference.maximumSingleAxleWeight"/1000 
                END as C20
                ,CASE WHEN mvcv.C21 IS NULL THEN 0 ELSE mvcv.C21 END as C21
                ,mvg.geom
    FROM        stg_bordenanalyse.matrixian_vehiclerestrictions mv
    LEFT JOIN   int_bordenanalyse.matrixian_vehiclerestrictions_constraints_verkeersborden mvcv
    ON          mv.id=mvcv.id
    LEFT JOIN   (SELECT  id,ST_Transform(ST_SetSRID(ST_GeomFromGeoJSON(geometry),4326),28992) as geom
                FROM    stg_bordenanalyse.matrixian_vehiclerestrictions_geom) as mvg
    ON          mv.id=mvg.id
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'matrixian_vehiclerestrictions_verkeersborden_geom'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'data integration'::VARCHAR);'''
cur.execute(sql_meta)

In [11]:
# create an empty table to store the signage mismatches
sql='''DROP TABLE IF EXISTS int_bordenanalyse.matrixian_mismatch;
CREATE TABLE IF NOT EXISTS int_bordenanalyse.matrixian_mismatch (
    matrixian_id    VARCHAR(255),
    city            VARCHAR(255),
    roadauthority   VARCHAR(255),
    matrixian_geom  GEOMETRY,
    road_id         INT,
    road_geom       GEOMETRY,
    road_restricted VARCHAR(25)
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'matrixian_mismatch'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'table initialization'::VARCHAR);'''
cur.execute(sql_meta)

In [12]:
# Check sign mismatch per matrixian polygon

# get list of matrixian id's
sql_matcodes=f'''
SELECT  id
FROM    int_bordenanalyse.matrixian_vehiclerestrictions_verkeersborden_geom mvvg
        ,(
            SELECT  ST_Union(ARRAY_AGG(geom)) as geom
            FROM    stg_ndw_wegen.cbs_provinciegrenzen_2023 c
            WHERE   statnaam IN {subselection_string}
            GROUP BY rubriek
        ) as p
WHERE ST_Within(mvvg.geom,p.geom)
ORDER BY id'''
cur.execute(sql_matcodes)
data=cur.fetchall()
data = [item for sublist in data for item in sublist]

for mat_code in data:
    sql=f"""
        INSERT INTO int_bordenanalyse.matrixian_mismatch(matrixian_id,city,roadauthority,matrixian_geom,road_id,road_geom,road_restricted)
        SELECT      mvvg.id as matrixian_id,mvvg."cityName" as city,"administrativeReference.municipalityName" as roadauthority
                    ,mvvg.geom as matrixian_geom
                    ,roads_that_should_be_restricted.id as road_id
                    ,roads_that_should_be_restricted.geom as road_geom
                    ,roads_that_are_restricted.road_restricted
        FROM        int_bordenanalyse.matrixian_vehiclerestrictions_verkeersborden_geom mvvg
        ,LATERAL    (
                        SELECT  wd.id, wd.geom
                        FROM    prd_ndw_wegen.nwb_wegvakken_directed wd
                        WHERE   ST_Within(wd.geom,mvvg.geom)
                        AND     wd.id>0
                    ) as roads_that_should_be_restricted
        LEFT JOIN LATERAL (
                        SELECT  b.road_id, 'yes' as road_restricted
                        -- auto is used as it is the least restrictive verhicle type, though vrachtauto may be more applicable
                        FROM    ST_Bereikbaarheidscheck(254441013,'''auto''',FALSE,mvvg.C17,mvvg.C18,mvvg.C19,mvvg.C20,mvvg.C21) b
                    ) as roads_that_are_restricted
        ON roads_that_should_be_restricted.id=roads_that_are_restricted.road_id
        WHERE       mvvg.id = '{mat_code}'
    ;"""
    cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'matrixian_mismatch'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

## Weergave matrixian in WebApp

In [ ]:
sql='''DROP TABLE IF EXISTS prd_bordenanalyse.matrixian_vehiclerestrictions_areas;
CREATE TABLE prd_bordenanalyse.matrixian_vehiclerestrictions_areas AS (
    SELECT      DISTINCT ON (mm.matrixian_id) matrixian_id, city, roadauthority, matrixian_geom as geom
    FROM        int_bordenanalyse.matrixian_mismatch mm
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_bordenanalyse'::VARCHAR,'matrixian_vehiclerestrictions_areas'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'data subset presentation'::VARCHAR);'''
cur.execute(sql_meta)

sql='''DROP TABLE IF EXISTS prd_bordenanalyse.matrixian_vehiclerestrictions_roads;
CREATE TABLE prd_bordenanalyse.matrixian_vehiclerestrictions_roads AS (
    SELECT      matrixian_id, road_id, road_geom
                ,CASE
                    WHEN road_restricted IS NULL THEN 'no'
                    ELSE road_restricted
                END as road_restricted
    FROM        int_bordenanalyse.matrixian_mismatch mm
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_bordenanalyse'::VARCHAR,'matrixian_vehiclerestrictions_roads'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'data subset presentation'::VARCHAR);'''
cur.execute(sql_meta)

# Missende verkeersborden
Door middel van een bereikbaarheidscheck kan worden gekeken of er verkeersborden missen: voor elk bord valt te checken of het wegvak waar het bord aan gekoppeld is via een omweg bereikbaar is, dan mist er waarschijnlijk een verkeersbord.
Deze check kan worden gedaan voor de type-restrictieborden (bijvoorbeeld C7: verboden voor vrachtauto's), maar ook voor de grootte/gewichtsbeperkingsborden, in het laatste geval wordt rekening gehouden met de exacte waarde van de beperking (bijvoorbeeld: een wegvak met aan beide kanten een lengtebeperking, maar met een verschillende getalswaarde)

In [ ]:
sql="""drop table if exists int_bordenanalyse.bereikbaarheid_typerestrictie;
CREATE TABLE int_bordenanalyse.bereikbaarheid_typerestrictie AS (
    WITH    bv3 as (
                -- subselectie van verkeersborden
                SELECT  bord_id, ndw_id, link_gevalideerd as road_id, gemeentenaam, straatnaam, geom
                        ,rvv_modelnummer
                        ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
                FROM    prd_ndw_borden.ndw_bebording_verrijkt_3
                WHERE   geldigheid='verbod'
            )
            -- diverse typen voertuigen
            ,auto_shouldbe_restricted as (
                SELECT  *
                FROM    bv3
                WHERE   rvv_modelnummer IN ('C6','C12')
            )
            ,auto_are_restricted as (
                SELECT  *, 'yes' as restricted
                FROM    ST_Bereikbaarheidscheck(254441013,'''auto''',FALSE,0,0,0,0,0)
            )
            ,auto_mismatch as (
                SELECT      auto_shouldbe_restricted.*
                FROM        auto_shouldbe_restricted 
                LEFT JOIN   auto_are_restricted
                ON          auto_are_restricted.road_id=abs(auto_shouldbe_restricted.road_id)
                WHERE       auto_are_restricted.restricted IS NULL
            )
            ,aanhanger_shouldbe_restricted as (
                SELECT  *
                FROM    bv3
                WHERE   rvv_modelnummer IN ('C6','C10','C12')
            )
            ,aanhanger_are_restricted as (
                SELECT  *, 'yes' as restricted
                FROM    ST_Bereikbaarheidscheck(254441013,'''auto''',TRUE,0,0,0,0,0)
            )
            ,aanhanger_mismatch as (
                SELECT      aanhanger_shouldbe_restricted.*
                FROM        aanhanger_shouldbe_restricted 
                LEFT JOIN   aanhanger_are_restricted
                ON          aanhanger_are_restricted.road_id=abs(aanhanger_shouldbe_restricted.road_id)
                WHERE       aanhanger_are_restricted.restricted IS NULL
            )
            ,vrachtauto_shouldbe_restricted as (
                SELECT  *
                FROM    bv3
                WHERE   rvv_modelnummer IN ('C6','C7','C7b','C12')
            )
            ,vrachtauto_are_restricted as (
                SELECT  *, 'yes' as restricted
                FROM    ST_Bereikbaarheidscheck(254441013,'''vrachtauto''',FALSE,0,0,0,0,4)
            )
            ,vrachtauto_mismatch as (
                SELECT      vrachtauto_shouldbe_restricted.*
                FROM        vrachtauto_shouldbe_restricted 
                LEFT JOIN   vrachtauto_are_restricted
                ON          vrachtauto_are_restricted.road_id=abs(vrachtauto_shouldbe_restricted.road_id)
                WHERE       vrachtauto_are_restricted.restricted IS NULL
            )
            ,bus_shouldbe_restricted as (
                SELECT  *
                FROM    bv3
                WHERE   rvv_modelnummer IN ('C6','C7a','C7b','C12')
            )
            ,bus_are_restricted as (
                SELECT  *, 'yes' as restricted
                FROM    ST_Bereikbaarheidscheck(254441013,'''bus''',FALSE,0,0,0,0,0)
            )
            ,bus_mismatch as (
                SELECT      bus_shouldbe_restricted.*
                FROM        bus_shouldbe_restricted 
                LEFT JOIN   bus_are_restricted
                ON          bus_are_restricted.road_id=abs(bus_shouldbe_restricted.road_id)
                WHERE       bus_are_restricted.restricted IS NULL
            )
            ,trekker_shouldbe_restricted as (
                SELECT  *
                FROM    bv3
                WHERE   rvv_modelnummer IN ('C6','C8','C9','C12')
            )
            ,trekker_are_restricted as (
                SELECT  *, 'yes' as restricted
                FROM    ST_Bereikbaarheidscheck(254441013,'''trekker''',FALSE,0,0,0,0,0)
            )
            ,trekker_mismatch as (
                SELECT      trekker_shouldbe_restricted.*
                FROM        trekker_shouldbe_restricted 
                LEFT JOIN   trekker_are_restricted
                ON          trekker_are_restricted.road_id=abs(trekker_shouldbe_restricted.road_id)
                WHERE       trekker_are_restricted.restricted IS NULL
            )
    SELECT  bord_id::varchar,ndw_id,rvv_modelnummer,gemeentenaam,straatnaam,geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,'Wegvak_bereikbaar_auto' as code
            ,'Achterliggend wegvak bereikbaar voor auto''s ondanks verbodsbord' as beschrijving
    FROM    auto_mismatch
    UNION ALL
    SELECT  bord_id::varchar,ndw_id,rvv_modelnummer,gemeentenaam,straatnaam,geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,'Wegvak_bereikbaar_aanhanger' as code
            ,'Achterliggend wegvak bereikbaar voor auto''s met aanhanger ondanks verbodsbord' as beschrijving
    FROM    aanhanger_mismatch
    UNION ALL
    SELECT  bord_id::varchar,ndw_id,rvv_modelnummer,gemeentenaam,straatnaam,geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,'Wegvak_bereikbaar_vrachtauto' as code
            ,'Achterliggend wegvak bereikbaar voor vrachtauto''s ondanks verbodsbord' as beschrijving
    FROM    vrachtauto_mismatch
    UNION ALL
    SELECT  bord_id::varchar,ndw_id,rvv_modelnummer,gemeentenaam,straatnaam,geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,'Wegvak_bereikbaar_bus' as code
            ,'Achterliggend wegvak bereikbaar voor bussen ondanks verbodsbord' as beschrijving
    FROM    bus_mismatch
    UNION ALL
    SELECT  bord_id::varchar,ndw_id,rvv_modelnummer,gemeentenaam,straatnaam,geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,'Wegvak_bereikbaar_trekker' as code
            ,'Achterliggend wegvak bereikbaar voor trekkers ondanks verbodsbord' as beschrijving
    FROM    trekker_mismatch
);"""
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'bereikbaarheid_typerestrictie'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

In [ ]:
sql="""drop table if exists int_bordenanalyse.bereikbaarheid_waarderestrictie;
CREATE TABLE int_bordenanalyse.bereikbaarheid_waarderestrictie AS (
    WITH    bv3 as (
                -- subselectie van verkeersborden
                SELECT  bord_id, ndw_id, link_gevalideerd as road_id, gemeentenaam, straatnaam, geom
                        ,rvv_modelnummer, tekst_waarde::FLOAT as tekst_waarde
                        ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
                FROM    prd_ndw_borden.ndw_bebording_verrijkt_3
                WHERE   geldigheid='verbod'
                AND     tekst_waarde IS NOT NULL
                AND     link_gevalideerd IS NOT NULL -- if this is NULL, the restriction won't have an effect anyway
            )
            ,c17_bv3 as (
                SELECT  *
                FROM    bv3
                WHERE   bv3.rvv_modelnummer = 'C17'
            )
            ,c17_distinct as (
                SELECT  DISTINCT ON (tekst_waarde) tekst_waarde
                FROM    c17_bv3
            )
            ,c17_mismatch as (
                SELECT  c17_bv3.*
                FROM    c17_distinct
                LEFT JOIN   c17_bv3
                ON      c17_distinct.tekst_waarde=c17_bv3.tekst_waarde
                LEFT JOIN LATERAL   (
                    SELECT  *, c17_distinct.tekst_waarde as waarde, 'yes' as restricted
                    FROM    ST_Bereikbaarheidscheck(254441013,'''auto''',FALSE,c17_distinct.tekst_waarde+0.1,0,0,0,0)
                ) c17_are_restricted
                ON      (c17_distinct.tekst_waarde=c17_are_restricted.waarde AND abs(c17_bv3.road_id)=c17_are_restricted.road_id)
                WHERE   c17_are_restricted.restricted IS NULL
            )
            ,c18_bv3 as (
                SELECT  *
                FROM    bv3
                WHERE   bv3.rvv_modelnummer = 'C18'
            )
            ,c18_distinct as (
                SELECT  DISTINCT ON (tekst_waarde) tekst_waarde
                FROM    c18_bv3
            )
            ,c18_mismatch as (
                SELECT  c18_bv3.*
                FROM    c18_distinct
                LEFT JOIN   c18_bv3
                ON      c18_distinct.tekst_waarde=c18_bv3.tekst_waarde
                LEFT JOIN LATERAL   (
                    SELECT  *, c18_distinct.tekst_waarde as waarde, 'yes' as restricted
                    FROM    ST_Bereikbaarheidscheck(254441013,'''auto''',FALSE,0,c18_distinct.tekst_waarde+0.1,0,0,0)
                ) c18_are_restricted
                ON      (c18_distinct.tekst_waarde=c18_are_restricted.waarde AND abs(c18_bv3.road_id)=c18_are_restricted.road_id)
                WHERE   c18_are_restricted.restricted IS NULL
            )
            ,c19_bv3 as (
                SELECT  *
                FROM    bv3
                WHERE   bv3.rvv_modelnummer = 'C19'
            )
            ,c19_distinct as (
                SELECT  DISTINCT ON (tekst_waarde) tekst_waarde
                FROM    c19_bv3
            )
            ,c19_mismatch as (
                SELECT  c19_bv3.*
                FROM    c19_distinct
                LEFT JOIN   c19_bv3
                ON      c19_distinct.tekst_waarde=c19_bv3.tekst_waarde
                LEFT JOIN LATERAL   (
                    SELECT  *, c19_distinct.tekst_waarde as waarde, 'yes' as restricted
                    FROM    ST_Bereikbaarheidscheck(254441013,'''auto''',FALSE,0,0,c19_distinct.tekst_waarde+0.1,0,0)
                ) c19_are_restricted
                ON      (c19_distinct.tekst_waarde=c19_are_restricted.waarde AND abs(c19_bv3.road_id)=c19_are_restricted.road_id)
                WHERE   c19_are_restricted.restricted IS NULL
            )
            ,c20_bv3 as (
                SELECT  *
                FROM    bv3
                WHERE   bv3.rvv_modelnummer = 'C20'
            )
            ,c20_distinct as (
                SELECT  DISTINCT ON (tekst_waarde) tekst_waarde
                FROM    c20_bv3
            )
            ,c20_mismatch as (
                SELECT  c20_bv3.*
                FROM    c20_distinct
                LEFT JOIN   c20_bv3
                ON      c20_distinct.tekst_waarde=c20_bv3.tekst_waarde
                LEFT JOIN LATERAL   (
                    SELECT  *, c20_distinct.tekst_waarde as waarde, 'yes' as restricted
                    FROM    ST_Bereikbaarheidscheck(254441013,'''auto''',FALSE,0,0,0,c20_distinct.tekst_waarde+0.1,0)
                ) c20_are_restricted
                ON      (c20_distinct.tekst_waarde=c20_are_restricted.waarde AND abs(c20_bv3.road_id)=c20_are_restricted.road_id)
                WHERE   c20_are_restricted.restricted IS NULL
            )
            ,c21_bv3 as (
                SELECT  *
                FROM    bv3
                WHERE   bv3.rvv_modelnummer = 'C21'
            )
            ,c21_distinct as (
                SELECT  DISTINCT ON (tekst_waarde) tekst_waarde
                FROM    c21_bv3
            )
            ,c21_mismatch as (
                SELECT  c21_bv3.*
                FROM    c21_distinct
                LEFT JOIN   c21_bv3
                ON      c21_distinct.tekst_waarde=c21_bv3.tekst_waarde
                LEFT JOIN LATERAL   (
                    SELECT  *, c21_distinct.tekst_waarde as waarde, 'yes' as restricted
                    FROM    ST_Bereikbaarheidscheck(254441013,'''auto''',FALSE,0,0,0,0,c21_distinct.tekst_waarde+0.1)
                ) c21_are_restricted
                ON      (c21_distinct.tekst_waarde=c21_are_restricted.waarde AND abs(c21_bv3.road_id)=c21_are_restricted.road_id)
                WHERE   c21_are_restricted.restricted IS NULL
            )
    SELECT  bord_id::VARCHAR, ndw_id, rvv_modelnummer, gemeentenaam, straatnaam, geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,concat('Wegvak_bereikbaar_','c17','_',tekst_waarde,'m') as code
            ,'Achterliggend wegvak bereikbaar ondanks lengtebeperkingsbord' as beschrijving
    FROM    c17_mismatch
    UNION ALL
    SELECT  bord_id::VARCHAR, ndw_id, rvv_modelnummer, gemeentenaam, straatnaam, geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,concat('Wegvak_bereikbaar_','c18','_',tekst_waarde,'m') as code
            ,'Achterliggend wegvak bereikbaar ondanks breedtebeperkingsbord' as beschrijving
    FROM    c18_mismatch
    UNION ALL
    SELECT  bord_id::VARCHAR, ndw_id, rvv_modelnummer, gemeentenaam, straatnaam, geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,concat('Wegvak_bereikbaar_','c19','_',tekst_waarde,'m') as code
            ,'Achterliggend wegvak bereikbaar ondanks hoogtebeperkingsbord' as beschrijving
    FROM    c19_mismatch
    UNION ALL
    SELECT  bord_id::VARCHAR, ndw_id, rvv_modelnummer, gemeentenaam, straatnaam, geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,concat('Wegvak_bereikbaar_','c20','_',tekst_waarde,'t') as code
            ,'Achterliggend wegvak bereikbaar ondanks aslastbeperkingsbord' as beschrijving
    FROM    c20_mismatch
    UNION ALL
    SELECT  bord_id::VARCHAR, ndw_id, rvv_modelnummer, gemeentenaam, straatnaam, geom
            ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            ,concat('Wegvak_bereikbaar_','c21','_',tekst_waarde,'t') as code
            ,'Achterliggend wegvak bereikbaar ondanks gewichtsbeperkingsbord' as beschrijving
    FROM    c21_mismatch
);"""
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'bereikbaarheid_waarderestrictie'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

# Verkeersbesluiten

In [2]:
# Check of alle borden (minimaal 1) entry in de koppeltabel met verkeersbesluiten hebben
sql='''DROP TABLE IF EXISTS int_bordenanalyse.koop_unmatched;
CREATE TABLE int_bordenanalyse.koop_unmatched AS (
    WITH    
        bv3 as (
            SELECT  bord_id::varchar, ndw_id, rvv_modelnummer, gemeentenaam, straatnaam, geom
                    ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            FROM    prd_ndw_borden.ndw_bebording_verrijkt_3
            WHERE   rvv_modelnummer NOT IN ('C13','C14','C15','C16')
        )
        ,vb as (
            -- 1 waarde per bord, met hoogste betrouwbaarheid en meest recent
            SELECT      DISTINCT ON (ndw_id) ndw_id, koop_id
            FROM        prd_ndw_borden.verkeersbesluiten_koppeltabel 
            ORDER BY    ndw_id, betrouwbaarheid DESC, datum_koppeling DESC
        )
    SELECT      bv3.*
    FROM        bv3
    LEFT JOIN   vb
    ON          bv3.ndw_id=vb.ndw_id
    WHERE       vb.koop_id IS NULL
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'koop_unmatched'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

# Bijhouden van huidige, nieuwe, bekende, verholpen issues
- Huidig: detectie uit huidige bordenbestand
- Bekend: detectie uit voorgaande bordenbestanden / historie
- Nieuw: huidige issues die niet voorkomen in bekend
- Verholpen: bekende issues die niet voorkomen in huidig

In [9]:
# Combinatietabel met alle fouten die gedetecteerd zijn in de (huidige versie van de) data
sql='''drop table if exists int_bordenanalyse.issues_huidig;
CREATE TABLE int_bordenanalyse.issues_huidig AS (
    WITH tekstwaarde_foutief AS (
            SELECT      tf.id as bord_id
                        ,bv3.ndw_id
                        ,'Onbekende_getalwaarde' as code
                        ,'Onbekende getalwaarde voor beperkingsbord' as beschrijving
                        ,bv3.rvv_modelnummer
                        ,bv3.gemeentenaam
                        ,bv3.straatnaam
                        ,bv3.geom
                        ,bv3.organisation_id, bv3.organisation_type_based_on_road, bv3.organisation_id_based_on_road, bv3.organisation_name_based_on_road
            FROM        int_bordenanalyse.tekstwaarde_foutief as tf
            LEFT JOIN   prd_ndw_borden.ndw_bebording_verrijkt_3 as bv3
            ON          tf.id = bv3.bord_id
        )
        ,matrixian_mismatch AS (
            SELECT      DISTINCT ON (mm.matrixian_id) matrixian_id as bord_id
                        , matrixian_id as ndw_id
                        ,'Matrixian_mismatch' as code
                        ,'Wegvakken binnen matrixian beperkingsgebied toch bereikbaar' as beschrijving
                        ,mm.city as gemeentenaam
                        ,'' as straatnaam
                        ,mm.roadauthority as organisation_id, '' as organisation_type_based_on_road, '' as organisation_id_based_on_road, '' as organisation_name_based_on_road
                        ,ST_Centroid(mm.matrixian_geom) as geom
            FROM        int_bordenanalyse.matrixian_mismatch as mm
        )
        ,union_fouten_db as (
            SELECT  bord_id::varchar, ndw_id, rvv_modelnummer, code, beschrijving, gemeentenaam, straatnaam, geom
                    ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            FROM    tekstwaarde_foutief
            UNION
            SELECT  bord_id::varchar, ndw_id, rvv_modelnummer, code, beschrijving, gemeentenaam, straatnaam, geom
                    ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            FROM    int_bordenanalyse.ndw_wvk_foutief
        )
        ,union_fouten_overig as (
            SELECT  bord_id::varchar, ndw_id, rvv_modelnummer
                    ,'verkeersbesluit_ontbreekt' as code
                    ,'Er is geen verkeersbesluit gekoppeld aan dit bord' as beschrijving
                    ,gemeentenaam, straatnaam, geom
                    ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            FROM    int_bordenanalyse.koop_unmatched
        )
        ,union_fouten_bereikbaarheid as (
            SELECT  bord_id::varchar, ndw_id, 'matrixian' as rvv_modelnummer, code, beschrijving, gemeentenaam, straatnaam, geom
                    ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            FROM    matrixian_mismatch
            UNION
            SELECT  bord_id::varchar, ndw_id, rvv_modelnummer, code, beschrijving, gemeentenaam, straatnaam, geom
                    ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            FROM    int_bordenanalyse.bereikbaarheid_typerestrictie
            UNION
            SELECT  bord_id::varchar, ndw_id, rvv_modelnummer, code, beschrijving, gemeentenaam, straatnaam, geom
                    ,organisation_id, organisation_type_based_on_road, organisation_id_based_on_road, organisation_name_based_on_road
            FROM    int_bordenanalyse.bereikbaarheid_waarderestrictie
        )
        ,union_fouten as (
            SELECT  *
            FROM    union_fouten_db
            UNION   
            SELECT  *
            FROM    union_fouten_overig
            UNION  
            SELECT  *
            FROM    union_fouten_bereikbaarheid
            --Borden die niet juist gekoppeld zijn, zullen altijd bereikbaarheidsfouten geven
            --WHERE   bord_id NOT IN (
            --    SELECT  bord_id
            --    FROM    union_fouten_db
            --)
        )
    SELECT  *
            ,now() as detectie_datum
    FROM    union_fouten
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'issues_huidig'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

In [10]:
# init tabel als hij niet bestaat
sql='''CREATE TABLE IF NOT EXISTS prd_bordenanalyse.issues_bekend AS (
    SELECT  *
            ,'' as status
            ,detectie_datum as mutatie_datum
    FROM    int_bordenanalyse.issues_huidig
    LIMIT   0
);'''
cur.execute(sql)
sql_alter='''DO $$ 
    BEGIN
        BEGIN
            ALTER TABLE prd_bordenanalyse.issues_bekend ADD COLUMN issue_id SERIAL PRIMARY KEY;
        EXCEPTION
            WHEN invalid_table_definition THEN 
            WHEN duplicate_column THEN
        END;
    END;
$$;'''
cur.execute(sql_alter)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_bordenanalyse'::VARCHAR,'issues_bekend'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'table initialization'::VARCHAR);'''
cur.execute(sql_meta)

In [11]:
# nieuwe issues = subset van huidig die niet voorkomt in bekend
sql='''drop table if exists int_bordenanalyse.issues_nieuw;
CREATE TABLE int_bordenanalyse.issues_nieuw AS (
    SELECT  *
    FROM    int_bordenanalyse.issues_huidig as h
    WHERE NOT EXISTS (
        SELECT  *
        FROM    prd_bordenanalyse.issues_bekend as b
        WHERE   h.bord_id = b.bord_id AND h.code = b.code
        AND     status != 'verholpen_in_bron'
    )
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'issues_nieuw'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

In [12]:
# verholpen issues = subset van bekend die niet voorkomt in huidig
sql='''drop table if exists int_bordenanalyse.issues_verholpen;
CREATE TABLE int_bordenanalyse.issues_verholpen AS (
    SELECT  *
    FROM    prd_bordenanalyse.issues_bekend as b
    WHERE NOT EXISTS (
        SELECT  *
        FROM    int_bordenanalyse.issues_huidig as h
        WHERE   h.bord_id = b.bord_id AND h.code = b.code
    )
    AND     b.status != 'verholpen_in_bron'
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'int_bordenanalyse'::VARCHAR,'issues_verholpen'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'calculation'::VARCHAR);'''
cur.execute(sql_meta)

In [14]:
# Update bekende issues door de nieuwe issues toe te voegen en de verholpen en/of oudere issues te markeren
sql_ins='''
INSERT INTO prd_bordenanalyse.issues_bekend(bord_id,ndw_id,rvv_modelnummer,code,beschrijving,gemeentenaam,straatnaam,detectie_datum,mutatie_datum,status,geom)
SELECT      n.bord_id,n.ndw_id,n.rvv_modelnummer,n.code,n.beschrijving,n.gemeentenaam,n.straatnaam,n.detectie_datum
            ,detectie_datum as mutatie_datum
            ,'nieuw' as status
            ,n.geom
FROM        int_bordenanalyse.issues_nieuw as n;'''
cur.execute(sql_ins)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_bordenanalyse'::VARCHAR,'issues_bekend'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'insert new records'::VARCHAR);'''
cur.execute(sql_meta)

sql_upd='''
UPDATE      prd_bordenanalyse.issues_bekend as ib
SET         status = 'verholpen_in_bron'
            ,mutatie_datum = now()
FROM        int_bordenanalyse.issues_verholpen as iv
WHERE       ib.bord_id = iv.bord_id AND ib.code = iv.code;'''
cur.execute(sql_upd)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_bordenanalyse'::VARCHAR,'issues_bekend'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'update records'::VARCHAR);'''
cur.execute(sql_meta)

sql_upd2='''
UPDATE      prd_bordenanalyse.issues_bekend as ib
SET         status = 'bekend'
            ,mutatie_datum = now()
WHERE       detectie_datum < (now() - '7 day'::interval)
AND         status = 'nieuw';'''
cur.execute(sql_upd2)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_bordenanalyse'::VARCHAR,'issues_bekend'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'update records'::VARCHAR);'''
cur.execute(sql_meta)

# Weergave in dashboard
- Gebaseerd op issues_bekend, maar met opgemaakte kolommen en benaming

In [2]:
# verholpen issues = subset van bekend die niet voorkomt in huidig
sql='''DROP TABLE IF EXISTS prd_bordenanalyse.issues_bekend_dashboard;
CREATE TABLE prd_bordenanalyse.issues_bekend_dashboard as (
    SELECT  issue_id,bord_id,ndw_id,status
            ,CASE
                WHEN rvv_modelnummer IN ('C17','C18','C19','C20','C21') THEN 'Restrictie ' || rvv_modelnummer::text
                ELSE 'Verbod ' || rvv_modelnummer::text
            END as rvv_modelnaam
            ,CASE
                WHEN length(rvv_modelnummer)<3 THEN 'https://stbereikbaarheid001.blob.core.windows.net/mapsprites/'||'C0'||SUBSTRING (rvv_modelnummer, 2)||'.svg'
                WHEN length(rvv_modelnummer)=3 AND (SUBSTRING (rvv_modelnummer, 3, 1) = 'a' OR SUBSTRING (rvv_modelnummer, 3, 1) = 'b') THEN 'https://stbereikbaarheid001.blob.core.windows.net/mapsprites/'||'C0'||SUBSTRING (rvv_modelnummer, 2)||'.svg'
                ELSE 'https://stbereikbaarheid001.blob.core.windows.net/mapsprites/'||rvv_modelnummer||'.svg'
            END as pictogram
            ,beschrijving,beschrijving as beschrijving_lang
            ,detectie_datum as datum_gedetecteerd
            ,CASE
                WHEN status = 'verholpen_in_bron' THEN mutatie_datum
                ELSE NULL
            END as datum_opgelost
            ,CASE
                WHEN organisation_type_based_on_road = 'G' THEN 'Gemeente '||organisation_name_based_on_road
                WHEN organisation_type_based_on_road = 'P' THEN 'Provincie '||organisation_name_based_on_road
                WHEN organisation_type_based_on_road = 'W' THEN 'Waterschap '||organisation_name_based_on_road
                WHEN organisation_type_based_on_road = 'R' THEN 'Rijkswaterstaat '||organisation_name_based_on_road
                ELSE organisation_name_based_on_road
            END as organisatienaam
            ,CASE 
                WHEN pg.statnaam in ('Noord-Holland','Flevoland') THEN 'RDT Noord-Holland en Flevoland'
                WHEN pg.statnaam in ('Utrecht') THEN 'RDT Midden-Nederland'
                ELSE 'Overig'
            END as rdt
    FROM    prd_bordenanalyse.issues_bekend ib
    LEFT JOIN stg_ndw_wegen.cbs_gemeentegrenzen_2023 gg
    ON      ib.gemeentenaam = gg.statnaam
    LEFT JOIN stg_ndw_wegen.cbs_provinciegrenzen_2023 pg
    ON      ST_Within(gg.geom,pg.geom)
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_bordenanalyse'::VARCHAR,'issues_bekend_dashboard'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'display table'::VARCHAR);'''
cur.execute(sql_meta)

In [3]:
# verholpen issues = subset van bekend die niet voorkomt in huidig
sql='''DROP TABLE IF EXISTS prd_bordenanalyse.bebording_dashboard;
CREATE TABLE prd_bordenanalyse.bebording_dashboard as (
    SELECT  bord_id, ndw_id
            ,CASE
                WHEN organisation_type_based_on_road = 'G' THEN 'Gemeente '||organisation_name_based_on_road
                WHEN organisation_type_based_on_road = 'P' THEN 'Provincie '||organisation_name_based_on_road
                WHEN organisation_type_based_on_road = 'W' THEN 'Waterschap '||organisation_name_based_on_road
                WHEN organisation_type_based_on_road = 'R' THEN 'Rijkswaterstaat '||organisation_name_based_on_road
                ELSE organisation_name_based_on_road
            END as organisatienaam
            ,CASE 
                WHEN pg.statnaam in ('Noord-Holland','Flevoland') THEN 'RDT Noord-Holland en Flevoland'
                WHEN pg.statnaam in ('Utrecht') THEN 'RDT Midden-Nederland'
                ELSE 'Overig'
            END as rdt
            ,CASE
                WHEN length(rvv_modelnummer)<3 THEN 'https://stbereikbaarheid001.blob.core.windows.net/mapsprites/'||'C0'||SUBSTRING (rvv_modelnummer, 2)||'.svg'
                WHEN length(rvv_modelnummer)=3 AND (SUBSTRING (rvv_modelnummer, 3, 1) = 'a' OR SUBSTRING (rvv_modelnummer, 3, 1) = 'b') THEN 'https://stbereikbaarheid001.blob.core.windows.net/mapsprites/'||'C0'||SUBSTRING (rvv_modelnummer, 2)||'.svg'
                ELSE 'https://stbereikbaarheid001.blob.core.windows.net/mapsprites/'||rvv_modelnummer||'.svg'
            END as pictogram
            ,CASE
                WHEN rvv_modelnummer IN ('C17','C18','C19','C20','C21') THEN 'Restrictie ' || rvv_modelnummer::text
                ELSE 'Verbod ' || rvv_modelnummer::text
            END as rvv_modelnaam
    FROM    prd_ndw_borden.ndw_bebording_verrijkt_3 bv3
    LEFT JOIN stg_ndw_wegen.cbs_gemeentegrenzen_2023 gg
    ON      bv3.gemeentenaam = gg.statnaam
    LEFT JOIN stg_ndw_wegen.cbs_provinciegrenzen_2023 pg
    ON      ST_Within(gg.geom,pg.geom)
);'''
cur.execute(sql)
sql_meta='''CALL update_metadata('rdt_dev'::VARCHAR,'prd_bordenanalyse'::VARCHAR,'bebording_dashboard'::VARCHAR,NOW(),'RDT_Bordenanalyse'::VARCHAR,'display table'::VARCHAR);'''
cur.execute(sql_meta)